# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import json

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tarinkot,32.6268,65.8733,27.06,17,0,1.84,AF,1682584722
1,1,grytviken,-54.2811,-36.5092,1.02,82,58,4.75,GS,1682584723
2,2,port elizabeth,-33.9180,25.5701,23.94,82,20,12.35,ZA,1682584723
3,3,namtsy,62.7161,129.6658,1.88,91,100,2.79,RU,1682584724
4,4,argo,37.6333,22.7333,18.21,45,3,1.99,GR,1682584725


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x = 'Lng',
    y = 'Lat',
    size = 'Humidity',
    color = 'City',
    geo = True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500
    )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"]>15) & (city_data_df["Max Temp"]<37) & (city_data_df["Wind Speed"]<10) & (city_data_df["Wind Speed"]>5) & (city_data_df["Cloudiness"]<=2)]

# Drop any rows with null values
idea_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,chu,43.5983,73.7614,25.44,22,0,9.15,KZ,1682584731
26,26,taoudenni,22.6783,-3.9836,34.92,7,0,9.29,ML,1682584738
27,27,akjoujt,19.6969,-14.4164,30.21,26,0,5.40,MR,1682584739
134,134,torchiarolo,40.4842,18.0525,16.76,50,1,6.33,IT,1682584813
157,157,pervomayskoye,43.9190,46.7102,22.08,54,0,7.93,RU,1682584831
165,165,aswan,24.0934,32.9070,33.61,13,0,5.14,EG,1682584837
300,300,puerto real,36.5282,-6.1901,28.67,38,0,5.66,ES,1682584935
322,322,tamanrasset,22.7850,5.5228,26.95,12,0,8.23,DZ,1682584953
383,383,pirallahi,40.4723,50.3330,20.08,68,0,6.56,AZ,1682584996
402,402,ulladulla,-35.3500,150.4667,16.54,76,0,5.51,AU,1682585010


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
14,chu,KZ,43.5983,73.7614,22,
26,taoudenni,ML,22.6783,-3.9836,7,
27,akjoujt,MR,19.6969,-14.4164,26,
134,torchiarolo,IT,40.4842,18.0525,50,
157,pervomayskoye,RU,43.9190,46.7102,54,
165,aswan,EG,24.0934,32.9070,13,
300,puerto real,ES,36.5282,-6.1901,38,
322,tamanrasset,DZ,22.7850,5.5228,12,
383,pirallahi,AZ,40.4723,50.3330,68,
402,ulladulla,AU,-35.3500,150.4667,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "apiKey":geoapify_key       
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = row['Lat']
    Longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
chu - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
akjoujt - nearest hotel: فندق فاطمة
torchiarolo - nearest hotel: Perla Hotel
pervomayskoye - nearest hotel: Европа
aswan - nearest hotel: Yaseen hotel
puerto real - nearest hotel: Hostal Bahía de Cádiz
tamanrasset - nearest hotel: فندق الأمان
pirallahi - nearest hotel: No hotel found
ulladulla - nearest hotel: Hotel Marlin
ndioum - nearest hotel: No hotel found
bushehr - nearest hotel: هتل رستوران سیراف
bowen - nearest hotel: Whitsunday Sands Resort


,City,Country,Lat,Lng,Humidity,Hotel Name
14,chu,KZ,43.5983,73.7614,22,No hotel found
26,taoudenni,ML,22.6783,-3.9836,7,No hotel found
27,akjoujt,MR,19.6969,-14.4164,26,فندق فاطمة
134,torchiarolo,IT,40.4842,18.0525,50,Perla Hotel
157,pervomayskoye,RU,43.9190,46.7102,54,Европа
165,aswan,EG,24.0934,32.9070,13,Yaseen hotel
300,puerto real,ES,36.5282,-6.1901,38,Hostal Bahía de Cádiz
322,tamanrasset,DZ,22.7850,5.5228,12,فندق الأمان
383,pirallahi,AZ,40.4723,50.3330,68,No hotel found
402,ulladulla,AU,-35.3500,150.4667,76,Hotel Marlin


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    x = 'Lng',
    y = 'Lat',
    color = 'City',
    size = 'Humidity',
    geo = True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)